#### 예제

1. csv 폴더 안 card 폴더에 있는 '오프라인.csv' 파일 로드
2. column의 이름을 변경 -> ['기준년월', '업종', '성별', '연령대', '총이용건수', '총이용금액']
3. 결측치가 존재하는가 ?
4. 기준년월 데이터를 시계열 데이터로 변경
5. 남, 여 성별을 기준으로 연도별 건당 이용금액의 평균이 몇인가?


In [81]:
import pandas as pd

offline = pd.read_csv('../../csv/card/오프라인.csv', encoding= 'CP949') # 한글이름 파일 로드가 UTF-8에서 어려우니 encoding을 CP949로 바꿨다.

In [82]:
offline.columns = ['기준년월', '업종','성별', '연령대', '총이용건수', '총이용금액'] # column의 이름을 한번에 변경하기 위해 list를 대입했다

In [83]:
offline.isna().any() # 결측치를 확인하기 위해 isna().any()를 사용하여 결측치가 존재하는 지 여부를 확인했다.

기준년월     False
업종       False
성별       False
연령대      False
총이용건수    False
총이용금액    False
dtype: bool

In [85]:
offline['기준년월'] = pd.to_datetime(offline['기준년월'], format= '%Y%m') # 기준년월을 시계열 타입으로 바꾸기 위해 pandas 내의 to_datetime() 함수를 사용했다.
                                                                            # 여기 inplace가 없는 이유는 pandas에서 로드했기 때문에 기준이 되는 data가 없음

In [86]:
offline['건당 이용금액'] = offline['총이용금액'] / offline['총이용건수'] # 5번 목표를 위해 '건당 이용금액' 파생변수를 생성해 offline에 새로운 column으로 삽입했다.

In [87]:
offline['연도'] = offline['기준년월'].map( # 5번 목표를 위해 '기준년월' 데이터에서 '연도' 파생변수를 생성해야 했고, 이를 위해 map() 함수와 lambda 함수를 이용해 치환하여 생성했다.
    lambda x : x.strftime('%Y') # 나 이거 처음에 생각 안 났어, 이거 복습 해야해.
)

# offline['연도'] = offline['기준년월'].dt.strftime('%Y') 이것도 됨 :>

In [88]:
offline # 생성된 파생 변수들이 잘 적용되었는 지 확인을 위한 구문

,기준년월,업종,성별,연령대,총이용건수,총이용금액,건당 이용금액,연도
0,2019-01-01,백화점,1,1,1,7.600000e+03,7600.000000,2019
1,2019-01-01,백화점,1,10,379,1.945872e+07,51342.269129,2019
2,2019-01-01,백화점,1,20,45173,4.747711e+09,105100.635357,2019
3,2019-01-01,백화점,1,30,271858,2.791826e+10,102694.279892,2019
4,2019-01-01,백화점,1,40,391109,3.791149e+10,96933.301601,2019
...,...,...,...,...,...,...,...,...
2643,2021-09-01,편의점,2,50,844081,6.283353e+09,7444.016988,2021
2644,2021-09-01,편의점,2,60,243768,1.997442e+09,8194.028646,2021
2645,2021-09-01,편의점,2,70,40523,3.524203e+08,8696.795894,2021
2646,2021-09-01,편의점,2,80,6349,5.333488e+07,8400.516932,2021


In [95]:
flag = offline['연령대'] == 0
offline[flag]

,기준년월,업종,성별,연령대,총이용건수,총이용금액,건당 이용금액,연도
10,2019-01-01,백화점,여성,0,2,128200.0,64100.000000,2019
31,2019-01-01,할인점,여성,0,1,19800.0,19800.000000,2019
51,2019-01-01,슈퍼마켓,여성,0,23,731100.0,31786.956522,2019
71,2019-01-01,편의점,여성,0,5,31680.0,6336.000000,2019
128,2019-02-01,슈퍼마켓,여성,0,5,248820.0,49764.000000,2019
...,...,...,...,...,...,...,...,...
2598,2021-09-01,할인점,여성,0,6,160420.0,26736.666667,2021
2608,2021-09-01,슈퍼마켓,남성,0,27,1481300.0,54862.962963,2021
2618,2021-09-01,슈퍼마켓,여성,0,63,851350.0,13513.492063,2021
2628,2021-09-01,편의점,남성,0,31,210240.0,6781.935484,2021


In [91]:
for i in range(len(offline['성별'])):
    if offline['성별'][i] == 1:
        offline['성별'][i] = '남성'
    else:
        offline['성별'][i] = '여성' # 이거 생각이 안 났었고, 결국 for문을 사용해 성별코드 1과 2를 각각 '남성', '여성'으로 치환했다.


C:\Users\USER\AppData\Local\Temp\ipykernel_1736\1001603131.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  offline['성별'][i] = '남성'
C:\Users\USER\AppData\Local\Temp\ipykernel_1736\1001603131.py:3: SettingWithCopyWarning: 
A value is tryin

In [92]:
offline[['성별','연도', '건당 이용금액']].groupby(['성별', '연도']).mean() # 생성된 파생 변수를 이용해 groupby()를 사용해 5번 목표를 달성했다.

건당 이용금액
성별 연도                
남성 2019  39184.349662
   2020  44009.589591
   2021  46948.320084
여성 2019  38107.838544
   2020  41389.763130
   2021  43306.318972

In [93]:
offline.pivot_table( # 생성된 파생 변수를 이용해 pivot_table()을 사용해 5번 목표를 달성했다.
    index = '연도',
    columns= '성별',
    values= '건당 이용금액',
    aggfunc= 'mean'
)



성별,남성,여성
연도,,
2019,39184.349662,38107.838544
2020,44009.589591,41389.763130
2021,46948.320084,43306.318972
